In [ ]:
!nvidia-smi

Wed Jun  1 08:48:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install -q sentence-transformers

     |████████████████████████████████| 4.2 MB 14.2 MB/s 
     |████████████████████████████████| 596 kB 78.6 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 56.4 MB/s 
     |████████████████████████████████| 346 kB 14.0 MB/s 
     |████████████████████████████████| 212 kB 66.5 MB/s 
     |████████████████████████████████| 140 kB 56.3 MB/s 
     |████████████████████████████████| 1.1 MB 74.1 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 127 kB 83.3 MB/s 
     |████████████████████████████████| 144 kB 85.9 MB/s 
     |████████████████████████████████| 271 kB 96.8 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is

In [ ]:
import os, sys
import random
from pathlib import Path
from tqdm.notebook import tqdm
from typing import Dict


import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from datasets import load_metric

from transformers import RobertaForMaskedLM, RobertaTokenizerFast, RobertaTokenizer
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput

from sentence_transformers import InputExample
from sentence_transformers import models, SentenceTransformer
from sentence_transformers import losses
from sentence_transformers.util import cos_sim
from sentence_transformers.evaluation import LabelAccuracyEvaluator
from sentence_transformers.losses.TripletLoss import TripletDistanceMetric

# Prepare Dataset

## Load Dataframe dataset

In [ ]:
train_df = pd.read_pickle('/content/drive/MyDrive/NLP/AI4Code/dataset/exp_5/clean_train_dataset.pkl')
train_df

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,00001756c60be8,1862f0a6,code,# this python 3 environment comes with many he...,0,945aea18,NaN,0.0
1,00001756c60be8,2a9e43d6,code,import numpy as np import pandas as pd import ...,2,945aea18,NaN,0.034483
2,00001756c60be8,038b763d,code,import warnings warnings.filterwarnings('ignore'),4,945aea18,NaN,0.068966
3,00001756c60be8,2eefe0ef,code,matplotlib.rcparams.update({'font.size': 14}),6,945aea18,NaN,0.103448
4,00001756c60be8,0beab1cd,code,"def evaluate_preds(train_true_values, train_pr...",8,945aea18,NaN,0.137931
...,...,...,...,...,...,...,...,...
6055708,fffe1d764579d5,0d770d6b,markdown,removing the outliers,43,3c40bfa6,NaN,0.597222
6055709,fffe1d764579d5,d45ddc62,markdown,dimensionality curse,33,3c40bfa6,NaN,0.458333
6055710,fffe1d764579d5,1a63248d,markdown,bangalore house price prediction,0,3c40bfa6,NaN,0.0
6055711,fffe1d764579d5,a8ffc8b4,markdown,we have achieved accuracy in predicting the pr...,69,3c40bfa6,NaN,0.958333


In [ ]:
val_df = pd.read_pickle('/content/drive/MyDrive/NLP/AI4Code/dataset/exp_5/clean_val_dataset.pkl')
val_df

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,0002115f48f982,18281c6c,code,import numpy as np # linear algebra import pan...,1,272b483a,NaN,0.111111
1,0002115f48f982,e3b6b115,code,df = pd.read_csv('../input/metadata_train.csv'...,2,272b483a,NaN,0.222222
2,0002115f48f982,4a044c54,code,df.head(),3,272b483a,NaN,0.333333
3,0002115f48f982,365fe576,code,#let's check if targets are consistent within ...,4,272b483a,NaN,0.444444
4,0002115f48f982,a3188e54,code,"sns.countplot(x='target',data=targets) # it sh...",5,272b483a,NaN,0.555556
...,...,...,...,...,...,...,...,...
314928,fff06cc23780af,f8135651,markdown,lowering the case,14,7b4c5375,NaN,0.17284
314929,fff06cc23780af,b61ba8ec,markdown,testing the model on test set,60,7b4c5375,NaN,0.740741
314930,fff06cc23780af,e98b7e0d,markdown,the aim of this notebook is to predict if twee...,2,7b4c5375,NaN,0.024691
314931,fff06cc23780af,f31fa490,markdown,having created this notebook from scratch star...,80,7b4c5375,NaN,0.987654


## Create cellid--source dictionary

In [ ]:
train_dict_cellid_source = dict(zip(train_df['cell_id'].values, train_df['source'].values))
val_dict_cellid_source = dict(zip(val_df['cell_id'].values, val_df['source'].values))

In [ ]:
def generate_triplet(df):
    triplets = []
    count = 0

    for id, df_tmp in tqdm(df.groupby('id')):
        df_tmp_markdown = df_tmp[df_tmp['cell_type'] == 'markdown']
        df_tmp_code = df_tmp[df_tmp['cell_type'] == 'code']

        df_tmp_code_rank = df_tmp_code['rank'].values
        df_tmp_code_cellid = df_tmp_code['cell_id'].values

        for cell_id, rank in df_tmp_markdown[['cell_id', 'rank']].values:
            labels = np.array([r == (rank+1) for r in df_tmp_code_rank]).astype('int')

            pos_position = np.where(labels == 1)[0]

            if len(pos_position):
                pos_position = pos_position[0]
                pos_cellid = df_tmp_code_cellid[pos_position]
                pos_label = labels[pos_position]
                # triplets.append([cell_id, pos_cellid, int(pos_label)])

                neg_positions = [x for x in range(0, len(labels)) if x != pos_position]

                if len(neg_positions):
                    # neg_ranks = df_tmp_code_rank[neg_positions].tolist()
                    # fil = list(filter(lambda x: x > rank, neg_ranks))
                    # if len(fil):
                    #     hard_neg_rank = min(fil)
                    #     hard_neg_cellid = df_tmp_code.loc[df_tmp_code['rank'] == hard_neg_rank, 'cell_id'].values[0]
                    #     hard_neg_label = 0
                    #     triplets.append([cell_id, pos_cellid, hard_neg_cellid])
                    # else:
                    #     neg_position = random.choice(neg_positions)
                    #     neg_cellid = df_tmp_code_cellid[neg_position]
                    #     triplets.append([cell_id, pos_cellid, neg_cellid])

                    neg_positions = random.choices(neg_positions, k=5)
                    neg_cellids = df_tmp_code_cellid[neg_positions]
                    for neg_cellid in neg_cellids:
                        triplets.append([cell_id, pos_cellid, neg_cellid])
    
    return triplets

## Take fraction of dataset for test running

In [ ]:
# train_df = train_df.loc[:600000]
# train_df

In [ ]:
# val_df = val_df.loc[:30000]
# val_df

In [ ]:
train_triplets = generate_triplet(train_df)
val_triplets = generate_triplet(val_df)

  0%|          | 0/132361 [00:00<?, ?it/s]

  0%|          | 0/6895 [00:00<?, ?it/s]

In [ ]:
train_triplets[:100]

[['21616367', '86497fe1', '1862f0a6'],
 ['21616367', '86497fe1', '9a78ab76'],
 ['21616367', '86497fe1', 'd9dced8b'],
 ['21616367', '86497fe1', '2eefe0ef'],
 ['21616367', '86497fe1', '8ca8392c'],
 ['fcb6792d', '5bf9ca51', 'bd94f005'],
 ['fcb6792d', '5bf9ca51', '06365725'],
 ['fcb6792d', '5bf9ca51', '76512d50'],
 ['fcb6792d', '5bf9ca51', 'd9dced8b'],
 ['fcb6792d', '5bf9ca51', '59959af5'],
 ['63c26fa2', '62638fba', '59959af5'],
 ['63c26fa2', '62638fba', 'bd94f005'],
 ['63c26fa2', '62638fba', '0e7c906e'],
 ['63c26fa2', '62638fba', '23783525'],
 ['63c26fa2', '62638fba', '781bbf3c'],
 ['4bb2e30a', 'bd94f005', '6b5664c7'],
 ['4bb2e30a', 'bd94f005', '6b5664c7'],
 ['4bb2e30a', 'bd94f005', '5bf9ca51'],
 ['4bb2e30a', 'bd94f005', '1862f0a6'],
 ['4bb2e30a', 'bd94f005', '5bf9ca51'],
 ['a6357f7e', 'ff7c44ed', '59959af5'],
 ['a6357f7e', 'ff7c44ed', '80151ab7'],
 ['a6357f7e', 'ff7c44ed', '781bbf3c'],
 ['a6357f7e', 'ff7c44ed', 'bd94f005'],
 ['a6357f7e', 'ff7c44ed', '9a78ab76'],
 ['45082c89', '781bbf3c',

## Define custom Dataset for Huggingface Trainer

In [ ]:
train_samples = []
for triplet in train_triplets:
    markdown_text = train_dict_cellid_source[triplet[0]]
    pos_code_text = train_dict_cellid_source[triplet[1]]
    neg_code_text = train_dict_cellid_source[triplet[2]]
    train_samples.append(InputExample(
        texts=[markdown_text, pos_code_text, neg_code_text]))

In [ ]:
val_samples = []
for triplet in val_triplets:
    markdown_text = val_dict_cellid_source[triplet[0]]
    pos_code_text = val_dict_cellid_source[triplet[1]]
    neg_code_text = val_dict_cellid_source[triplet[2]]
    val_samples.append(InputExample(
        texts=[markdown_text, pos_code_text, neg_code_text]))

## Training dataloader

In [ ]:
BATCH_SIZE = 16

train_dataloader = DataLoader(train_samples, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=0, drop_last=False)

val_dataloader = DataLoader(val_samples, batch_size=BATCH_SIZE, shuffle=False,
                              num_workers=0, drop_last=False)

# Sentence Transformer Model

In [ ]:
# bert = models.Transformer("/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_mlm/checkpoint-160000", max_seq_length=256)
# pooler = models.Pooling(
#     bert.get_word_embedding_dimension(),
#     pooling_mode_mean_tokens=True,
# )

# model = SentenceTransformer(modules=[bert, pooler])
model = SentenceTransformer('/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_task/trial_2/')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Freeze some layer for faster training

In [ ]:
# for name, module in model.named_modules():
#     print(name)

In [ ]:
# model[0].auto_model.encoder

In [ ]:
# for param in model[0].auto_model.encoder.parameters():
#     param.requires_grad = False

# Loss function

In [ ]:
loss = losses.TripletLoss(model=model)

# Train Model

In [ ]:
from sentence_transformers.evaluation import TripletEvaluator

In [ ]:
evaluator = TripletEvaluator.from_input_examples(val_samples)

In [23]:
epochs = 1
# warmup_steps = int(len(train_dataloader) * epochs * 0.1)
warmup_steps = 0

model.fit(
    train_objectives=[(train_dataloader, loss)],
    epochs=epochs,
    evaluator=evaluator,
    evaluation_steps=20000,
    warmup_steps=warmup_steps,
    output_path='/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_task/trial_3',
    checkpoint_path='/content/drive/MyDrive/NLP/AI4Code/pretrained/exp_5/pretrained_task/trial_3',
    optimizer_params={'lr': 5e-6},
    checkpoint_save_steps=40000,
    checkpoint_save_total_limit=2,
    use_amp=True,
    show_progress_bar=True,
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/469110 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# optimizer_params = {'lr': 1e-5}
# warmup_steps=0